In [9]:
import os
import numpy as np
import poisson as ps
import numpy.matlib
from sklearn.datasets import load_svmlight_file

## Data load

In [6]:
data_folder = '../../data'

In [7]:
problem_name='a9a'
results = {problem_name: {}}
W, y = load_svmlight_file(os.path.join(data_folder, problem_name))
y = y.reshape(-1, 1)
W = W.toarray()

## Data preprocessing

In [10]:
# multiplicative multiplication of the structure of the recognizable structure
W = numpy.matlib.repmat(y, 1, W.shape[1]) * W
# summation of feature descriptions
sA = np.sum(W, 1)
# if the sum of the string is negative, invert it
W[sA < 0, :] = -W[sA < 0, :]
# zero row deletion
W = W[sA != 0, :]
# W = scipy.sparse.csr_matrix(W)

Bias = 1
b = (Bias * y).squeeze()
b = np.abs(b)
if (b.any == 0):
    print(' Input parameter y error')

y = b
N, n = W.shape

## Parameters initialization

In [11]:
lam =  np.sqrt(N)/2
#first set of parameters
Mf = max(2 / np.sqrt(y))
nu = 3
I = np.eye(n)
Iv = np.ones(n)
#running parameters
x0 = np.ones(n) / n
terminate_tol = 1e-10
M = ps.poisson(W, y, lam, x0)[0] / lam

FW_params={
    'iter_FW': 50000,
    'line_search_tol': 1e-10,
}


sc_params={
    #parameters for SCOPT
    'iter_SC': 1000,
    'Lest': 'estimate', #estimate L
    'use_two_phase': True,
    #FISTA parameters
    'fista_type': 'mfista',
    'fista_tol': 1e-5,
    'fista_iter': 1000,
    #Conjugate Gradient Parameters
    'conj_grad_tol': 1e-2,
    'conj_grad_iter': 100,
}

## Auxilary functions

In [12]:
func_x = lambda x: ps.poisson(W, y, lam, x)
grad_x = lambda x, dot_product: ps.grad_poisson(W, y, lam, x, dot_product)
grad_beta = lambda x, s, beta, dot_product, dot_product_s: ps.grad_poisson(W, y, lam, (1 - beta) * x + beta * x, (1 - beta) * dot_product + beta * dot_product_s)
hess_mult_x = lambda x, dot_product: ps.hess_mult(W, y, x, dot_product)
hess_mult_vec_x = lambda s, dot_product: ps.hess_mult_vec(W, y, s, dot_product)
extra_func = lambda x: W @ x
linear_orcale = lambda grad: ps.linear_oracle_full_simplex(grad, M)
prox_func = lambda x, L: np.maximum(x, np.zeros(len(x))) #used for SCOPT

## Run SCOPT

In [ ]:
 x, alpha_hist, Q_hist, time_hist, grad_hist = sc.scopt(func_x,
          grad_x,
          hess_mult_x,
          hess_mult_vec_x,
          Mf,
          nu,
          prox_func,
          x0,  
          sc_params,                                              
          eps=terminate_tol,                                              
          print_every=10)
    
results[example]['scopt'] = {
    'x': x,
    'alpha_hist': alpha_hist,
    #'Gap_hist': Gap_hist,
    'Q_hist': Q_hist,
    'time_hist': time_hist,
    'grad_hist': grad_hist
}